In [1]:
import os
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import sounddevice as sd
import sys

from obspy.core import AttribDict
from obspy.core import Stats

from obspy.core import Trace, Stream
from obspy.io.segy.segy import SEGYBinaryFileHeader
from obspy.io.segy.segy import SEGYTraceHeader

In [9]:
path = os.getcwd()
command = [
    'avconv',
    '-i', os.path.join(path, 'dat/ur-soundoff.mp3'),
    '-f', 's16le',
    '-acodec', 'pcm_s16le',
    '-ar', '44100', # ouput will have 44100 Hz
    '-ac', '2', # stereo (set to '1' for mono)
    '-'
]

pipe = subprocess.Popen(command, 
                        stdout=subprocess.PIPE, 
                        bufsize=10**8)
stdoutdata = pipe.stdout.read()
audio_array = np.fromstring(stdoutdata, dtype="int16")

In [3]:
audio_array.shape

(342701620,)

In [4]:
ns = 2001
#print(audio_array.shape[0] / ns)

pad = np.pad(audio_array, (0, int(np.ceil(audio_array.shape[0] / ns)) * ns - audio_array.shape[0]), 'constant')

ur = np.reshape(pad, (ns, -1), order='F')
print(ur.shape)

(2001, 171266)


In [11]:
head = 'Undersampled Radio - Seismic Soundoff'
head = head.encode('utf-8')

stream = Stream()

for i in range(32767):
    # build audio into trace data
    data = ur[:, i]
    data = np.require(data, dtype=np.float32)
    trace = Trace(data=data)

    # Attributes in trace.stats will overwrite everything in trace.stats.segy.trace_header
    trace.stats.delta = 0.004

    if not hasattr(trace.stats, 'segy.trace_header'):
        trace.stats.segy = {}
    trace.stats.segy.trace_header = SEGYTraceHeader()
    trace.stats.segy.trace_header.trace_sequence_number_within_line = i + 1
    trace.stats.segy.trace_header.receiver_group_elevation = 0

    # Add trace to stream
    stream.append(trace)


stream.stats = AttribDict()
stream.stats.textual_file_header = head
stream.stats.binary_file_header = SEGYBinaryFileHeader()
stream.stats.binary_file_header.trace_sorting_code = 5



stream.write("dat/test.sgy", format="SEGY", data_encoding=1)

!segyread tape=dat/test.sgy > dat/test.su
print('\n')
!surange < dat/test.su

3200+0 records in
6+1 records out
3240 bytes (3.2 kB, 3.2 KiB) copied, 0.00120783 s, 2.7 MB/s


/bin/sh: 1: cannot open test.su: No such file


In [12]:
!suxwigb < dat/test.su n2=300


suxwigb: subcalls.c: efwrite: fwrite only 187 items of 2001
